In [1]:
%load_ext autoreload
%autoreload 2


In [ ]:
from uuid import uuid4
import os
from steel import Steel
from steel.types.session import Session as SteelSession

from dotenv import load_dotenv

steel_base_url = "http://localhost:3000/"

steel_api_key="hello"

load_dotenv()

client = Steel(
    # This is the default and can be omitted
    # steel_api_key=os.environ.get("STEEL_API_KEY"),
    steel_api_key=steel_api_key,
    base_url=steel_base_url,
)

# const sessionId = uuidv4(); // '9b1deb4d-3b7d-4bad-9bdd-2b0d7b3dcb6d'

# const browser = await chromium.connectOverCDP(
#     `wss://connect.steel.dev?apiKey=${process.env.STEEL_API_KEY}&sessionId=${sessionId}`
# );

# // Get session details
# const client = new Steel();
# const session = await client.sessions.retrieve(sessionId);
# console.log(`View session live at: ${session.sessionViewerUrl}`);
n_sessions = 1

sessions: list[SteelSession] = []
for i in range(n_sessions):
    session = client.sessions.create(
        #use_proxy=True,
        #solve_captcha=True,
        timeout=30 * 60 * 1000 # 30 minutes
    )
    sessions.append(session)


In [ ]:
client.sessions.list()

In [ ]:
client.sessions.retrieve(session.id)

In [ ]:
from patchright.async_api import async_playwright
from tqdm import tqdm
playwright = await async_playwright().start()


for session in tqdm(sessions):
    browser = await playwright.chromium.connect_over_cdp(
        #f'wss://{steel_base_url.replace("http://", "").replace("https://", "")}?apiKey={steel_api_key}'
        #f"ws://localhost:3000/devtools/browser/{session.id}"
        #f"ws://localhost:3000",
        #f"ws://0.0.0.0:3000/"
        #"f"ws://0.0.0.0:3000/?apiKey={steel_api_key}&sessionId={session.id}"
        f"ws://0.0.0.0:3000/devtools/browser/{session.id}"
    )
    page = await browser.new_page()



    await page.goto("https://www.google.com")
    await page.wait_for_load_state("networkidle")
    await page.screenshot(path="screenshot.png")
    await page.accessibility.snapshot()

In [ ]:
from typing import Any
from dotenv import load_dotenv
from patchright.async_api import async_playwright
from steel import Steel
from steel.types.session import Session as SteelSession
import os

# Load environment variables from .env file
load_dotenv()
# Connect Playwright to the Steel session
playwright = await async_playwright().start()

STEEL_API_KEY = os.getenv('STEEL_API_KEY')
STEEL_BASE_URL = "localhost:3000"



# Initialize Steel client with the API key from environment variables
client = Steel(steel_api_key=STEEL_API_KEY, base_url=f"http://{STEEL_BASE_URL}")


print("Creating Steel session...")
session = client.sessions.create()

print(f"""Session created successfully with Session ID: {session.id}.
You can view the session live at {session.session_viewer_url}
""")


async def get_playwright_page(session: SteelSession):
    browser: Browser = await playwright.chromium.connect_over_cdp(
        f"ws://{STEEL_BASE_URL}/devtools/browser/{session.id}"
    )

    print("Connected to browser via Playwright")

    # Create page at existing context to ensure session is recorded
    current_context = browser.contexts[0]
    if len(current_context.pages) == 0:
        page = await current_context.new_page()
    else:
        page =  current_context.pages[0]

    return page


released_session = client.sessions.release(session.id)
if released_session.success:
    print(f"Session {released_session.id} released successfully")
else:
    print(f"Session {released_session.id} release failed")

page = await get_playwright_page(session)

# Navigate to Hacker News and extract the top 5 stories
print("Navigating to Hacker News...")
await page.goto("https://news.ycombinator.com", wait_until="networkidle")


In [ ]:
released_session

In [14]:
for context in browser.contexts:
    for page in context.pages:
        print(page.url)


In [ ]:






async def extract_story_details(row: Any) -> dict[str, str]:
    title_element = row.locator(".titleline > a")
    title = await title_element.text_content()
    link = await title_element.get_attribute("href")

    points_element = row.locator("xpath=following-sibling::tr[1]").locator(".score")
    points = "0"
    if await points_element.count() > 0:
        points = (await points_element.text_content()).split()[0]

    return {
        "title": title,
        "link": link,
        "points": points
    }


# Find all story rows
# story_rows = await page.locator("tr.athing").all()
# story_rows = story_rows[:5]  # Get first 5 stories

# # Extract the top 5 stories using Playwright's locators
# print("\nTop 5 Hacker News Stories:")
# for i, row in enumerate(story_rows, 1):
#     story = await extract_story_details(row)
#     print(f"\n{i}. {story['title']}")
#     print(f"   Link: {story['link']}")
#     print(f"   Points: {story['points']}")



In [ ]:
if browser:
    await browser.close()
    print("Browser closed")

if session:
    print("Releasing session...")
    client.sessions.release(session.id)
    print("Session released")

    print("Done!")


In [ ]:
session = client.sessions.create()
client.sessions.release(session.id) 
_session = client.sessions.retrieve(session.id)